In [2]:
%load_ext cython

In [11]:
%%cython
import numpy as np
from mpire import WorkerPool

pool = WorkerPool(n_jobs=5)

cdef float epsilon, delta, LB, UB, mu, sig
cdef short n_max
cdef int n, t
cdef list X_t

epsilon = 0.01
delta = 0.05
n_max = 208
t0 = 6-1
def cB_n(n, sig, delta = 0.05, epsilon = 0.01):
    return sig * np.sqrt(2*np.log(3*n_max/delta)/(n+t0)**2) + 3*np.log(3*n_max/delta)/(n+t0)**2
# def cB_n(n, sig, delta = 0.05, epsilon = 0.01):
    # return sig * np.sqrt(2*np.log(3*n**2*np.pi**2/6/delta)/n**2) + 3*np.log(3*n**2*np.pi**2/6/delta)/n**2

def IEBLR(symulation, p1, p2, delta = 0.05, epsilon = 0.01, n_max = 191):
    LB = 0
    UB = 1
    t = 1
    X_t = []
    for n in range(1, n_max):
        X_t.extend(np.random.random(size=2*n - 1)/2)
        # X_t.extend(pool.map(symulation, [[p1,p2]]*t))
        mu = np.mean(X_t)  
        sig = np.std(X_t)
        cn = cB_n(n, sig)
        LB = max(LB, mu - cn)
        UB = min(UB, mu + cn)
        if UB - LB < 2*epsilon:
            break
    return mu

In [12]:
%%cython
import numpy as np
from mpire import WorkerPool


cdef float epsilon, delta, LB, UB, mu, sig
cdef short n_max
cdef int n, t
cdef list X_t

epsilon = 0.01
delta = 0.05
n_max = 208
t0 = 6-1
def cB_n(n, sig, delta = 0.05, epsilon = 0.01):
    return sig * np.sqrt(2*np.log(3*n_max/delta)/(n+t0)**2) + 3*np.log(3*n_max/delta)/(n+t0)**2
# def cB_n(n, sig, delta = 0.05, epsilon = 0.01):
    # return sig * np.sqrt(2*np.log(3*n**2*np.pi**2/6/delta)/n**2) + 3*np.log(3*n**2*np.pi**2/6/delta)/n**2

def IEBLR2(symulation, p1, p2, delta = 0.05, epsilon = 0.01, n_max = 191, n_jobs = 4):
    LB = 0
    UB = 1
    t = 1
    X_t = []
    # with WorkerPool(n_jobs=n_jobs) as pool:
    #  pool = WorkerPool(n_jobs=n_jobs)
    for n in range(1, n_max):
        X_t.extend(np.random.random(size=2*n - 1)/2)
            # X_t.extend(pool.map(symulation, [[p1, p2]]*t))
        mu = np.mean(X_t)  
        sig = np.std(X_t)
        cn = cB_n(n, sig)
        LB = max(LB, mu - cn)
        UB = min(UB, mu + cn)
        if UB - LB < 2*epsilon:
            break
        elif UB < 0.5:
            break
        elif LB > 0.5:
            break
    if LB > 0.5:
        return 1
    elif UB < 0.5:
        return 2
    elif mu > 0.5:
        return 1
    else:
        return 2

In [15]:
x = np.array([IEBLR(1,2,3) for i in range(10)])
print(np.mean(x))

0.24998819459352878


In [6]:
IEBLR(1,2,3)

Process ForkWorker-17:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Process ForkWorker-18:
  File "/home/olek/.local/lib/python3.10/site-packages/mpire/worker.py", line 164, in run
    func = self._get_func(additional_args)
  File "/home/olek/.local/lib/python3.10/site-packages/mpire/worker.py", line 251, in _get_func
    partial(self.map_params.func, *additional_args))
Traceback (most recent call last):
TypeError: the first argument must be callable
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Process ForkWorker-19:
  File "/home/olek/.local/lib/python3.10/site-packages/mpire/worker.py", line 164, in run
    func = self._get_func(additional_args)
  File "/home/olek/.local/lib/python3.10/site-packages/mpire/worker.py", line 251, in _get_func
    partial(self.map_params.func, *additional_args))
Traceback (most recent call last):
TypeError: the first ar

KeyboardInterrupt: 